# Tutorial 02: Decision Trees and Learning from the Data
  * You will learn how to create (learn, train) a decision tree model from the data.
  * We will try a characteristic approach to create an ML model.
  * We will explore basics of the package `sklearn`.
  * Starting with raw data, we will finish with a trained model and know, approximately how good is it.

In [ ]:
import math
import pandas as pd
import numpy as np

## Exploring the data

  * We will use the package `pandas` to load and explore data.
  * The dataset is from the server [kaggle.com](https://www.kaggle.com), where you can find [the description of the dataset and its particular features](https://www.kaggle.com/uciml/adult-census-income).

In [ ]:
data = pd.read_csv('adult-census.csv')

print(f'Shape of the dataset is {data.shape}\n')
display(data.head())

In [ ]:
display(data.info())
display(data.describe())

In [ ]:
print(f'Number of unique values in the columns:')
display(data.nunique()) # NaNs are not counted by default

### Missing values
  * Some features are apparently missing, but there are no NaNs.
  * Missing values are marked with a questionmark `?`.

In [ ]:
data = data.replace('?', np.nan)
# look at the change with .info()
data.info()

print(f'\nNumber of NaN values in the columns:')
data.isnull().sum(axis=0)

## First attempt to use the decision tree from `scikit-learn`

 * [Documentation](http://scikit-learn.org/stable/modules/tree.html) of decision tree from the library `scikit-learn`.
 * We now only try, whether our data is ready to use with this implementation.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# split the dataset into feature matrix and target vector (column income)
Xdata = data.iloc[:,:-1]
ydata = data.iloc[:,-1] 

The ML models from `scikit-learn` have uniform interface.
1. **Initialize the model class and set hyperparameters.** You can find possible settings in the documentation.

In [ ]:
dt = DecisionTreeClassifier()
dt.get_params()

2. **Learn the model on the data with method `.fit()`.**

In case of supervised models:
  * `X` should be array-like with shape $(n\_samples, n\_features)$ - e.g. `pandas.DataFrame` or `numpy.ndarray`
  * `y` should be array-like with shape $(n\_samples)$ - e.g. `pandas.Series` or `numpy.ndarray`

In [ ]:
# dt.fit(Xdata, ydata)

* **It does not work:** decision tree from `scikit-learn` cannot work with other than numeric features.
* `string` features can be:
    * **nominal** - Named data without order, also called **categorical**. Can be replaced with one-hot encoding, label encoding or dropped. 
    * **ordinal** - Ordered data. We can naturaly replace strings with numbers.
    
### Task
Is there any dependency between columns *education* and *education.num*?

In [ ]:
display(data[['education', 'education.num']].head(10))

### your code

---

In [ ]:
display(data.dtypes)
string_cols = data.select_dtypes(['object']).columns

print('\nThe following columns have string values:')
print(string_cols)
data[string_cols].nunique()

* All features other than *education* are **categorical**.
* We will use label encoding.
    * It can be done with `pandas` by changing column type to `category`.

In [ ]:
data[string_cols] = data[string_cols].astype('category').apply(lambda x: x.cat.codes)
data.info()

Does it work?

In [ ]:
display(data['native.country'].unique())

What happened with the missing values?

In [ ]:
data.occupation.unique()
data.groupby(["occupation"]).size() # there were 1843 NaNs in the column occupation

**Missing values were replaced with -1!**

There are many approaches how to handle missing values and none of them is generally correct. Replacing `NaN` with a single unique value is the most simple way, which is also used in practice.

In [ ]:
dt = DecisionTreeClassifier()
dt.get_params()

In [ ]:
Xdata = data.iloc[:,:-1]
ydata = data.iloc[:,-1]

# pro případ, kdy vysvětlována proměnná není v posledním sloupci
Xdata2 = data.drop(columns='income')
ydata2 = data.income

In [ ]:
# porovnání výsledků iloc a drop
print(Xdata.equals(Xdata2))
print(ydata.equals(ydata2))

In [ ]:
dt.fit(Xdata, ydata)

## Training the decision tree
  * The objective is to create a decision tree predicting *income* of given persons.
  * We can train different trees - with different values of parameters like `max_depth` and `criterion`. These parameters which can be tuned by the user are called hyperparameters.
  * To tune hyperparameters, we need to split our data into *train*, *validation* and *test* set.
      * The function `train_test_split` from `sklearn.model_selection` do just this.

In [ ]:
rd_seed = 333 # data is splitted randomly, defined seed is needed for result replication
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(Xdata, ydata, test_size=0.25, random_state=rd_seed) 
print("original data")
display(Xdata.shape)
display(ydata.shape)
print("train data")
display(Xtrain.shape)
display(ytrain.shape)
print("test data")
display(Xtest.shape)
display(ytest.shape)

In [ ]:
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=rd_seed) 
print('new train data')
display(Xtrain.shape)
display(ytrain.shape)
print('validation data')
display(Xval.shape)
display(yval.shape)

### Measuring training error with classification accuracy

In [ ]:
dt = DecisionTreeClassifier(max_depth=50)
dt.fit(Xtrain, ytrain)

* The decision tree is trained now.
* We can use method `.predict()` to obtain predictions.

In [ ]:
ypredicted = dt.predict(Xtrain)
ypredicted

Accuracy can be computed as

$$acc = \frac{\#correct}{N},$$

where $\#correct$ is number of correct predictions and $N$ si the number of samples.

In [ ]:
(ytrain - ypredicted).value_counts(normalize=True)

  * Train accuracy (for `max_depth` = 50) is almost 100%!
  * Accuracy can be also computed with function from `scikit-learn`.

In [ ]:
import sklearn.metrics as metrics
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, dt.predict(Xtrain))))

And test set accuracy:

In [ ]:
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, dt.predict(Xtest))))

### Hyperparameter tuning - `max_depth` a `criterion`

  * Let's start with `max_depth`.
  * We will use validation set.
  * We will explore values 1 to 100.

In [ ]:
val_acc = []
train_acc = []
depths = range(1,101)
for depth in depths:
    dt = DecisionTreeClassifier(max_depth=depth)
    dt.fit(Xtrain, ytrain)
    train_acc.append(metrics.accuracy_score(ytrain, dt.predict(Xtrain)))
    val_acc.append(metrics.accuracy_score(yval, dt.predict(Xval)))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(20,6))
plt.plot(depths, train_acc,'or-') #o: points, r: color, -: linking points with a line
plt.plot(depths, val_acc,'ob-')
plt.xlabel('max_depth')
plt.ylabel('accuracy')
plt.legend(['train', 'validation'])

In [ ]:
optimal_depth = np.argmax(val_acc)
print(f'optimal_depth: {optimal_depth}\naccuracy score (validation): {val_acc[optimal_depth - 1]:.6f}')

Let's compute **test accuracy** for the found value of `max_depth`:

In [ ]:
dt = DecisionTreeClassifier(max_depth=optimal_depth)
dt.fit(Xtrain, ytrain)
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, dt.predict(Xtest))))

### Grid search: tuning multiple hyperparameters

  * We want to try `max_depth` from interval $[1, 30]$ for both possible `criterion` values - 60 combinations.
  * We can use class `ParameterGrid`.

In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {
    'max_depth': range(1,31), 
    'criterion': ['entropy', 'gini']
}
param_comb = ParameterGrid(param_grid)
# for better understanding
[param_comb[i] for i in range(28, 32)]

In [ ]:
val_acc = []
train_acc = []
for params in param_comb:
    dt = DecisionTreeClassifier(max_depth=params['max_depth'], criterion=params['criterion'])
    dt.fit(Xtrain, ytrain)
    train_acc.append(metrics.accuracy_score(ytrain, dt.predict(Xtrain)))
    val_acc.append(metrics.accuracy_score(yval, dt.predict(Xval)))

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(train_acc,'or-')
plt.plot(val_acc,'ob-')
plt.xlabel('hyperparametr index')
plt.ylabel('accuracy')
plt.legend(['train', 'validation'])

In [ ]:
best_params = param_comb[np.argmax(val_acc)]
print(best_params)
print(val_acc[np.argmax(val_acc)])
print(train_acc[np.argmax(val_acc)])

In [ ]:
dt = DecisionTreeClassifier(**best_params) # two asterisks in Python are MIGHTY
dt.fit(Xtrain, ytrain)
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, dt.predict(Xtrain))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, dt.predict(Xval))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, dt.predict(Xtest))))

In [ ]:
# ** operator example
def add(a, b=0, c=0):
    print(a + b + c)

args = [2]
kwargs = {
    'b': 1,
    'c': -2
}

add(*args, **kwargs)

### TASK 1: Train decision tree on the Titanic data from previous tutorial (1 bonus point)

  * Choose your own hyperparameters for tuning.
  * Select the same ratio and `random_state` for splitting data with `train_test_split()` as above.

In [ ]:
titanic = pd.read_csv('titanic.csv', sep=';')
### your code

---
# Visualisation of decision tree

In [ ]:
simple_data = pd.read_csv('simple_data.csv', sep=';') # data from the lecture
display(simple_data.head(16)) 
dts = DecisionTreeClassifier(criterion='entropy', max_depth=2)
dts.fit(simple_data.iloc[:,1:],simple_data['T'])

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

plt.figure(figsize=(10,10))
plot_tree(dts, filled=True, feature_names=simple_data.iloc[:,1:].columns, class_names=True, rounded=True) ;